In [68]:
!pip install "stable-baselines3[extra]>=2.0.0a4"

import stable_baselines3
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.sac.sac import SAC
from stable_baselines3.ppo import MlpPolicy
import pickle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
def evaluate(model, vec_env, deterministic=True):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    episode_rewards = []
    trajectory = []
    done = False
    obs = vec_env.reset()
    while not done:
      # _states are only useful when using LSTM policies
      action, _states = model.predict(obs, deterministic=deterministic)
      # here, action, rewards and dones are arrays
      # because we are using vectorized env
      # also note that the step only returns a 4-tuple, as the env that is returned
      # by model.get_env() is an sb3 vecenv that wraps the >v0.26 API
      obs, reward, done, info = vec_env.step(action)
      episode_rewards.append(reward)
      trajectory.append((obs, action, reward))

    mean_episode_reward = np.mean(episode_rewards)
    print("Mean reward:", mean_episode_reward)

    return trajectory

env_id = "Acrobot-v1"
env = gym.make(env_id, render_mode="rgb_array")
model = PPO(MlpPolicy, env, verbose=0)
# Train the agent for 10000 steps
model.learn(total_timesteps=10000)

vec_env = model.get_env()

gym.wrappers.RecordVideo(vec_env, './videos')
trajectory = evaluate(model, vec_env, deterministic=True)

with open('/content/demo.pkl', 'wb') as f:
    pickle.dump(trajectory, f)

Mean reward: -1.0
